In [175]:
import pandas as pd
import requests

In [176]:
urls = {
    'internet':'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/retailindustry/datasets/retailsalesindexinternetsales/current/internetreferencetables.xlsx',
    'pounds':'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/retailindustry/datasets/poundsdatatotalretailsales/current/poundsdata.xlsx',
    # 'index':'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/retailindustry/datasets/retailsalesindexreferencetables/current/mainreferencetables.xlsx'
}

dropColumns = [
    'All retailing including automotive fuel',
    'Month as a % of Total',
    'Month as a % of Total excluding automotive fuel',
    'Total non-food stores',
    'Total Annual Sales for All Retailing including automotive fuel',
    'Total Annual Sales for All Retailing excluding automotive fuel'
]

In [177]:
# download each workbook
for _ in urls:
    with open(f'{_}.xlsx', 'wb') as output:
        output.write(requests.get(urls[_]).content)

In [178]:
# parse multi-sheet workbook to dataframe by selecting sheet
dfPounds = pd.ExcelFile('pounds.xlsx').parse('ValSAW')

# remove rows not containing data
dfPounds.columns = dfPounds.iloc[3]
dfPounds.columns.name = None
dfPounds = dfPounds.iloc[6:]
for _ in dropColumns:
    dfPounds = dfPounds.drop(_,axis=1)

# parse time period and set as index
dfPounds['Time Period'] = pd.to_datetime(dfPounds['Time Period'], format='%Y%b')
dfPounds = dfPounds.set_index('Time Period',drop=True)
dfPounds = dfPounds.loc['2008-01-01':]

# multiply values by number of weeks for month average
dfPounds.iloc[:,1:] = dfPounds.iloc[:,1:].multiply(dfPounds.loc[:, 'Number of Weeks'], axis="index")

In [179]:
# parse multi-sheet workbook to dataframe by selecting sheet
dfInternet = pd.ExcelFile('Internet.xlsx').parse('IntValSA')

# remove rows not containing data
dfInternet.columns = dfInternet.iloc[2]
dfInternet.columns.name = None
dfInternet = dfInternet.iloc[5:]

# parse time period and set as index
dfInternet['Time Period'] = pd.to_datetime(dfInternet['Time Period'], format='%Y %b')
dfInternet = dfInternet.set_index('Time Period',drop=True)
dfInternet = dfInternet.drop('Total of predominantly non-food stores [Note 2]',axis=1)

# change values from £m to £k matching dfPounds
dfInternet = dfInternet * 1000

# copy column names and add weeks column from dfPounds
dfInternet.columns = dfPounds.columns[1:-1]
dfInternet = dfInternet.join(dfPounds['Number of Weeks'])
col = dfInternet.pop('Number of Weeks')
dfInternet.insert(0, col.name, col)

# multiply values by number of weeks for month average
dfInternet.iloc[:,1:] = dfInternet.iloc[:,1:].multiply(dfInternet.loc[:, 'Number of Weeks'], axis="index")

In [180]:
# create dataframe for offline sales
dfOffline = dfPounds.iloc[:,1:] - dfInternet.iloc[:,1:]
dfOffline = dfOffline.join(dfPounds['Number of Weeks'])
col = dfOffline.pop('Number of Weeks')
dfOffline.insert(0, col.name, col)